In [1]:
# ETL PRoject

In [48]:
import pandas as pd
import psycopg2
import sqlalchemy
import numpy as np
import matplotlib as plt
from sqlalchemy import create_engine, inspect
from datetime import datetime

In [3]:
# Database connection 
pg_port = '5432'
pg_username = 'postgres'
pg_pw = 'postgres'
pg_db = 'movies'
postgresDSN = ('postgresql://{}:{}@{}:{}/{}'.format(pg_username, pg_pw, 'localhost', pg_port, pg_db))
engine = create_engine(postgresDSN)
cnx = engine.connect()

In [4]:
# create the database schema
# Query was run directly in postgres. Can also be run from jupyter notes

# #query = f""" 

# Drop Table IF EXISTS Movie_Actor CASCADE;  
# Drop Table IF EXISTS Ratings CASCADE;  
# Drop Table IF EXISTS Movies CASCADE;  
# Drop Table IF EXISTS Actors CASCADE;  

# CREATE TABLE  Actors  (
#     actor_id  INT   NOT NULL,
#      first_name  VARCHAR(255)   NOT NULL,
#      last_name  VARCHAR(255)   NOT NULL,
#      birth_name  VARCHAR(255),
#      date_of_birth  date,
#      place_of_birth  text,
#      date_of_death  date,
#      reason_of_death  text,
#     CONSTRAINT  pk_Actors  PRIMARY KEY (
#         actor_id 
#     )
# );

# CREATE TABLE  Movies  (
#     movie_id  INT   NOT NULL,
#     title  text   NOT NULL,
#     year  INT   NOT NULL,
#     genre  VARCHAR(255)   NOT NULL,
#     duration  INT,
#     language  text,
#     director  VARCHAR(255),
#     actors  text,
#     description  text,
#     avg_vote  float   NOT NULL,
#     budget  float,
#     gross_income  float,
#     critic_reviews  float,
#     CONSTRAINT  pk_Movies  PRIMARY KEY (
#         movie_id 
#     )
# );

# CREATE TABLE  Ratings  (
#     movie_id  INT   NOT NULL,
#     avg_vote_females  float,
#     avg_vote_males  float,
#     total_votes  INT   NOT NULL,
#     weighted_average_vote  float   NOT NULL,
#    CONSTRAINT  pk_Ratings  PRIMARY KEY (
#         movie_id 
#     )
# );

# CREATE TABLE  Movie_Actor  (
#     id  serial   NOT NULL,
#     movie_id  INT   NOT NULL,
#     actor_id  INT   NOT NULL,
#     role  text,
#     characters  text,
#    CONSTRAINT  pk_Movie_Actor  PRIMARY KEY (
#         id 
#     )
# );

# ALTER TABLE  Ratings  ADD CONSTRAINT  fk_Ratings_movie_id  FOREIGN KEY( movie_id )
# REFERENCES  Movies  ( movie_id );

# ALTER TABLE  Movie_Actor  ADD CONSTRAINT  fk_Movie_Actor_movie_id  FOREIGN KEY( movie_id )
# REFERENCES  Movies  ( movie_id );

# ALTER TABLE  Movie_Actor  ADD CONSTRAINT  fk_Movie_Actor_actor_id  FOREIGN KEY( actor_id )
# REFERENCES  Actors  ( actor_id );

# """

In [5]:
data_source = 'Resources/'

In [6]:
movies_csv = data_source + 'IMDB movies.csv'
actors_csv = data_source + 'IMDB names.csv'
ratings_csv = data_source + 'IMDB ratings.csv'
title_principals = data_source + 'IMDb title_principals.csv'

In [49]:
actors = pd.read_csv(actors_csv, low_memory=False)

In [50]:
actors.head(3)

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (hi...,2,0,1,2
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 196...,2,1,2,3
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,Bernard d'Ormale (16 August 1992 - present)\n...,4,3,1,1


In [51]:
actors.rename(columns={'imdb_name_id':'actor_id'}, inplace=True)

In [52]:
actors_cols = ['actor_id','name','birth_name','date_of_birth','place_of_birth','date_of_death','reason_of_death']

In [53]:
actors = actors[actors_cols]

In [54]:
actors['date_of_birth']= actors['date_of_birth'].astype(str)
actors['date_of_birth']= actors['date_of_birth'].str.split(" ", expand = True)
actors['date_of_birth'] = pd.to_datetime(actors['date_of_birth'], errors='coerce') 
actors['date_of_birth'].dropna()
actors['date_of_death']= actors['date_of_death'].astype(str)
actors['date_of_death']= actors['date_of_death'].str.split(" ", expand = True)
actors['date_of_death'] = pd.to_datetime(actors['date_of_death'], errors='coerce') 
actors.head(100)

,actor_id,name,birth_name,date_of_birth,place_of_birth,date_of_death,reason_of_death
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,1899-05-10,"Omaha, Nebraska, USA",1987-06-22,pneumonia
1,nm0000002,Lauren Bacall,Betty Joan Perske,1924-09-16,"The Bronx, New York City, New York, USA",2014-08-12,stroke
2,nm0000003,Brigitte Bardot,Brigitte Bardot,1934-09-28,"Paris, France",NaT,NaN
3,nm0000004,John Belushi,John Adam Belushi,1949-01-24,"Chicago, Illinois, USA",1982-03-05,acute cocaine and heroin intoxication
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,1918-07-14,"Uppsala, Uppsala län, Sweden",2007-07-30,natural causes
...,...,...,...,...,...,...,...
95,nm0000097,Pamela Anderson,Pamela Denise Anderson,1967-07-01,"Ladysmith, British Columbia, Canada",NaT,NaN
96,nm0000098,Jennifer Aniston,Jennifer Joanna Aniston,1969-02-11,"Sherman Oaks, California, USA",NaT,NaN
97,nm0000099,Patricia Arquette,Patricia T. Arquette,1968-04-08,"Chicago, Illinois, USA",NaT,NaN
98,nm0000100,Rowan Atkinson,Rowan Sebastian Atkinson,1955-01-06,"Consett, County Durham, England, UK",NaT,NaN


In [55]:
# Cleaning currency column
def clean_currency(x):
    """ If the value is a string with currency symbol, split the string and 
    return numeric value. Else return original input. 
    """
    if isinstance(x, str):
        try: 
            return x.split()[1]
        except IndexError as e:
            return x
    return(x)

In [56]:
# Clean date columns - reove string from text from date 
def clean_date(x):
    for i in str(x).split():
        if i.isdigit():
            return i
        if i.isalpha():
            pass
        else:
            return x

In [57]:
# Check actor id, remover 'nm' prefix and returns numeric portion. Else skips the row if its not in 'nm000123' format
def clean_id(x):
    if x[2:].isdigit() and x[0:2] == 'nm':
        return x[2:]
    else:
        pass

In [58]:
# Clean the data
actors['actor_id'] = actors['actor_id'].apply(lambda x: x.replace('nm', ''))
actors['first_name'] = actors.loc[:,'name'].apply(lambda x: x.split()[0])
actors['last_name'] = actors.loc[:,'name'].apply(lambda x: x.split()[1:])
actors['last_name'] = actors.loc[:,'last_name'].astype(str).apply(lambda x: x.strip('][').replace("'", ""))
actors['date_of_birth'] = actors.loc[:,'date_of_birth'].apply(lambda x: clean_date(x))
actors['date_of_death'] = actors.loc[:,'date_of_death'].apply(lambda x: clean_date(x))
actors = actors[['actor_id','first_name','last_name','birth_name','date_of_birth','place_of_birth','date_of_death','reason_of_death']]

In [59]:
actors.head(3)

,actor_id,first_name,last_name,birth_name,date_of_birth,place_of_birth,date_of_death,reason_of_death
0,0000001,Fred,Astaire,Frederic Austerlitz Jr.,1899-05-10,"Omaha, Nebraska, USA",1987-06-22,pneumonia
1,0000002,Lauren,Bacall,Betty Joan Perske,1924-09-16,"The Bronx, New York City, New York, USA",2014-08-12,stroke
2,0000003,Brigitte,Bardot,Brigitte Bardot,1934-09-28,"Paris, France",NaT,NaN


In [18]:
list(actors.columns)

['actor_id',
 'first_name',
 'last_name',
 'birth_name',
 'date_of_birth',
 'place_of_birth',
 'date_of_death',
 'reason_of_death']

### Movies Data 

In [19]:
movies = pd.read_csv(movies_csv, low_memory=False)

In [20]:
rename_cols = {
    'imdb_title_id':'movie_id',
    'worlwide_gross_income':'gross_income',
    'reviews_from_critics': 'critic_reviews'
}

In [21]:
movies.rename(columns=rename_cols, inplace=True)

In [22]:
movie_cols = ['movie_id','title','year','genre','duration','language','director','actors', \
             'description', 'avg_vote', 'critic_reviews']

In [23]:
movies = movies[movie_cols]

In [24]:
movies.head(3)

,movie_id,title,year,genre,duration,language,director,actors,description,avg_vote,critic_reviews
0,tt0000009,Miss Jerry,1894,Romance,45,None,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,2.0
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,None,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,7.0
2,tt0001892,Den sorte drøm,1911,Drama,53,NaN,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,2.0


In [25]:
movies['movie_id'] = movies['movie_id'].apply(lambda x: x.replace('tt', ''))
movies['year'] = movies.loc[:,'year'].apply(lambda x: clean_date(x))

In [26]:
movies.head(3)

,movie_id,title,year,genre,duration,language,director,actors,description,avg_vote,critic_reviews
0,0000009,Miss Jerry,1894,Romance,45,None,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,2.0
1,0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,None,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,7.0
2,0001892,Den sorte drøm,1911,Drama,53,NaN,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,2.0


In [27]:
list(movies.columns)

['movie_id',
 'title',
 'year',
 'genre',
 'duration',
 'language',
 'director',
 'actors',
 'description',
 'avg_vote',
 'critic_reviews']

### Ratings data

In [28]:
ratings = pd.read_csv(ratings_csv)

In [29]:
rename_cols = {
    'imdb_title_id':'movie_id',
    'females_allages_avg_vote':'avg_vote_females',
    'males_allages_avg_vote':'avg_vote_males'
}

In [30]:
ratings.rename(columns=rename_cols, inplace=True)

In [31]:
ratings_cols = ['movie_id','avg_vote_females','avg_vote_males','total_votes','weighted_average_vote']

In [32]:
ratings = ratings[ratings_cols]

In [33]:
ratings['movie_id'] = ratings['movie_id'].apply(lambda x: x.replace('tt', ''))

In [34]:
ratings.head(3)

,movie_id,avg_vote_females,avg_vote_males,total_votes,weighted_average_vote
0,0000009,6.0,6.2,154,5.9
1,0000574,6.2,6.1,589,6.1
2,0001892,5.7,5.9,188,5.8


In [35]:
list(ratings.columns)

['movie_id',
 'avg_vote_females',
 'avg_vote_males',
 'total_votes',
 'weighted_average_vote']

### movie_actors table

In [36]:
movie_actor = pd.read_csv(title_principals)

In [37]:
movie_actor.head(3)

,imdb_title_id,ordering,imdb_name_id,category,job,characters
0,tt0000009,1,nm0063086,actress,NaN,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]"
2,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor..."


In [38]:
rename_cols = {
    'imdb_title_id':'movie_id',
    'imdb_name_id':'actor_id',
    'category': 'role'
}

In [39]:
movie_actor.rename(columns=rename_cols, inplace=True)

In [40]:
movie_actor['movie_id'] = movie_actor['movie_id'].apply(lambda x: x.replace('tt', ''))
movie_actor['actor_id'] = movie_actor['actor_id'].apply(lambda x: x.replace('nm', ''))

In [41]:
movie_actor.head(3)

,movie_id,ordering,actor_id,role,job,characters
0,0000009,1,0063086,actress,NaN,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,0000009,2,0183823,actor,NaN,"[""Mr. Hamilton""]"
2,0000009,3,1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor..."


In [42]:
movie_actor = movie_actor[['movie_id','actor_id','role','characters']]

In [43]:
list(movie_actor.columns)

['movie_id', 'actor_id', 'role', 'characters']

### Load Sql Tables

In [44]:
# read the data to tables:
movies.to_sql('movies', schema='public', con=cnx, if_exists='append', index=False)

In [61]:
actors.to_sql('actors', schema='public', con=cnx, if_exists='append', index=False)

In [46]:
ratings.to_sql('ratings', schema='public', con=cnx, if_exists='append', index=False)

In [47]:
movie_actor.to_sql('movie_actor', schema='public', con=cnx, if_exists='append', index=False)